In [7]:
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, BatchNormalization, Input, Dropout, Activation, concatenate, GRU, Dropout
import keras.backend as K
from keras.optimizers import Adam, Nadam, SGD
import numpy as np

In [8]:
import os, sys, time, threading
import tensorflow as tf
import numpy as np
import pandas as pd
# Disable depreciation warnings and limit verbosity during training
from tensorflow.python.util import deprecation

In [9]:
RATE = 0.001
NUM_EPOCHS = 11
VALSPLIT = 0.0
np.random.seed(5)
TRAIN_BATCH_SIZE = 2024

In [10]:
X = pd.read_pickle("input/X_HB.pkl")[:]
X.drop(['PU','pt'],1,inplace=True)
mu,std = np.mean(X, axis=0), np.std(X, axis=0)
X = (X-mu)/std
Y = pd.read_pickle("input/Y_HB.pkl")[:][['genE']]

In [11]:
inputs_keras = Input(shape=(X.shape[1],), name='input')
h = Dropout(rate=RATE)(inputs_keras)
h = Dense(36, activation='relu')(h)
norm = Dropout(rate=RATE)(h)
h = Dense(11, activation = 'relu')(norm)
norm = Dropout(rate=RATE)(h)
h = Dense(3, activation = 'relu')(norm)
net_keras = Dense(1, activation='linear', name='output')(h)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model_keras = Model(inputs=inputs_keras, outputs=net_keras)
model_keras.compile(optimizer=Adam(), loss='mse')

In [13]:

#model_keras.fit(X.values, Y.values,batch_size=2024, epochs=NUM_EPOCHS, shuffle=True,validation_data=(X, Y))
idx = np.random.permutation(X.shape[0])
train_idx = idx[int(VALSPLIT*len(idx)):]
val_idx = idx[:int(VALSPLIT*len(idx))]
# model_keras.fit(X.values[train_idx], Y.values[train_idx],batch_size=2024, 
#                 epochs=NUM_EPOCHS, shuffle=True, validation_data = (X.values[val_idx], Y.values[val_idx]))

model_keras.fit(X.values[train_idx], Y.values[train_idx],batch_size=TRAIN_BATCH_SIZE, 
                epochs=NUM_EPOCHS, shuffle=True)#, validation_data = (X.values[val_idx], Y.values[val_idx]))

result = model_keras.predict(X, 10000)
print(result[:10])

Instructions for updating:
Use tf.cast instead.
Epoch 1/11
574212/574212 [==============================] - 2s 3us/step - loss: 49.9268
Epoch 2/11
574212/574212 [==============================] - 2s 3us/step - loss: 1.5127
Epoch 3/11
574212/574212 [==============================] - 2s 3us/step - loss: 1.1050
Epoch 4/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.8999
Epoch 5/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.7749
Epoch 6/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.7372
Epoch 7/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.6835
Epoch 8/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.6318
Epoch 9/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.6938
Epoch 10/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.6242
Epoch 11/11
574212/574212 [==============================] - 2s 3us/step - loss:

In [48]:
MODEL_DIR = 'gs://harrisgroup-ctpu/facile/model/'
DATA_DIR = 'gs://harrisgroup-ctpu/facile/data/'
TPU_NAME='uw-tpu'
ZONE_NAME='us-central1-b'
PROJECT_NAME = 'harrisgroup-223921'
NUM_ITERATIONS = 100000 # Number of iterations per TPU training loop
TRAIN_STEPS = 70000
NUM_SHARDS = 8 # Number of shards (TPU chips).
USE_TPU = True

lock = threading.Lock()

#tf.enable_eager_execution()

# DEFINE THE NETWORK

class ModelFacile(object):
    def __call__(self, inputs):
        net = tf.layers.dropout(inputs, rate = 0.001)
        net = tf.layers.dense(net, 36, activation = 'relu')
        norm = tf.layers.dropout(net, rate = 0.001)
        net = tf.layers.dense(norm, 11, activation = 'relu')
        norm = tf.layers.dropout(net, rate = 0.001)
        net = tf.layers.dense(norm, 3, activation = 'relu')
        return tf.layers.dense(net, 1, activation = 'linear', name='output')


def input_facile(train_x,train_y,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((tf.cast(train_x,tf.float32),tf.cast(train_y,tf.float32)))
    return dataset.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))

def model_fn(features, labels, mode, params):
    model = ModelFacile()

    if mode == tf.estimator.ModeKeys.PREDICT:
        predicted_values = model(features)
        #print("Shape of predicted values is ", tf.shape(predicted_values))
        predictions = {
            'probabilities': predicted_values,
        }
        return tf.contrib.tpu.TPUEstimatorSpec(mode, predictions=predictions)

    predicted_values = model(features)
    loss = tf.losses.mean_squared_error(labels=labels,predictions=predicted_values)

    if mode == tf.estimator.ModeKeys.TRAIN:
      optimizer = tf.train.AdagradOptimizer(learning_rate = 0.1)
      if USE_TPU:
        optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)
      train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
      return tf.contrib.tpu.TPUEstimatorSpec(mode, loss=loss, train_op=train_op)


# CREATE AND PREDICT WITH TPUS

def create_estimator(batch_size):
    print("Creating the estimator")
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
        TPU_NAME,
        zone=ZONE_NAME,
        project=PROJECT_NAME)

    run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        model_dir=MODEL_DIR,
        session_config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True),
        tpu_config=tf.contrib.tpu.TPUConfig(NUM_ITERATIONS, NUM_SHARDS))

    estimator = tf.contrib.tpu.TPUEstimator(
        model_fn=model_fn,
        use_tpu=USE_TPU,
        train_batch_size=batch_size,
        eval_batch_size=batch_size,
        predict_batch_size=batch_size,
        params={"data_dir": DATA_DIR},
        config=run_config)
    return estimator

def predict(data, batch_size,estimator, results=None, times=None, job_id=None):
    assert (results is None and times is None and job_id is None) or not (results is None or times is None or job_id is None)
    lock.acquire()
    

    print("Predicting")
    def predict_input_fn(params):
        batch_size = params["batch_size"]
        dataset_predict = tf.data.Dataset.from_tensor_slices(data.astype('float32'))
        return dataset_predict.batch(batch_size)

    predictions = estimator.predict(predict_input_fn)
    
    num_print = 10
    num = 0
    predicted_values = []
    start_time = time.time()
    for pred_dict in predictions:
        #print(pred_dict.keys())
        #print(pred_dict.values())
        break
        #predicted_values.append(list(pred_dict.values())[0][0])
        #num += 1
#         if (num >= num_print):
#             break
    #predicted_values =np.array(predicted_values)/np.sum(predicted_values)
    #print("Number of prediction is ",i)
    #print(predicted_values[:10])

    predict_time = time.time() - start_time
    lock.release()

    return predicted_values[:10],predict_time

In [44]:
#estimator = create_estimator(2024)
#estimator.train(input_fn=lambda params: input_facile(X,Y,params["batch_size"]),max_steps=TRAIN_STEPS)

In [51]:
time_list = []
tf.logging.set_verbosity(tf.logging.FATAL)
estimator = create_estimator(100000)
for N in [1000,10000,100000,600000]:
    start_time = time.time()
    predicted_values = predict(X[:N],1,estimator)
    time_in_s = time.time() - start_time
    print("Time used in the prediction is ", time_in_s)
    time_list.append(time_in_s/N)


Creating the estimator
Predicting
Time used in the prediction is  4.95343542098999
Predicting
Time used in the prediction is  8.588505983352661
Predicting
Time used in the prediction is  9.893390655517578
Predicting
Time used in the prediction is  16.089070796966553


In [52]:
print(time_list)

[0.00495343542098999, 0.0008588505983352661, 9.893390655517579e-05, 2.6815117994944256e-05]


In [27]:
(0.00019141989707946778*600000-27)/600000

0.00014641989707946777

In [19]:
[13.894327163696289/1000, 41.09413719177246/10000, 149.68007469177246/100000, 642.9999010562897/600000]

[0.013894327163696289,
 0.004109413719177246,
 0.0014968007469177247,
 0.0010716665017604828]

In [ ]:
predicted_values

1. TRAIN_STEPS*TRAIN_BATCH_SIZE < len(X)
2. Wrong learning rate kills everything

In [ ]:
np.set_printoptions(precision=3)
print(np.array(Y["genE"][:10]))

In [ ]:
print(result[:10])